# L3 Slice detection
### Last update: October 3, 2020
----

This IPython notebook works correctly on Google Colab. Since it requires Linux with a GPU accelerator material. It has not been tested on another environment.

To have the code running:
- Copy the "sarcopenia-ai-master" folder on Google Drive or anywhere accessible from this IPython notebook file.
- Make sure that the DCM file collections are copied in an accessible folder (i.e data) maintaining the structure (Patient_Folder/Study_Folder/Serie_Folder/Collection_of_dcm_files)
- Exexute the code cells related to the block "**Setting up the environnment**."
- If data is already in NII format, skip the the cells related to the block "**Data Pre-processing**".
- Execute the code cells related to the block "**Executing the algorithm**" to do prediction

The ouputs jpeg files (Slice and Frontal images) will be created in the working directory (i.e /content/gdrive/My Drive/sarcopenia-ai-master/).

Also, a excel file will all output details will be created (algo1_output.csv).




## Setting up the **environment**

In [ ]:
# Mount the Google drive folder so we can access the folder contanining the collection of DCMs
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Change the working directory to the project directory
%cd /content/gdrive/My Drive/sarcopenia-ai-master/

/content/gdrive/My Drive/sarcopenia-ai-master


In [ ]:
# Download Anaconda (uncomment the next line if necessary) and install that framework
#!wget https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh
!bash Anaconda3-5.2.0-Linux-x86_64.sh -bfp /usr/local

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: blas-1.0-mkl ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: intel-openmp-2018.0.0-8 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libgfortran-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: bzip2-1.0.6-h14c3975_5 ...
installing: expat-2.2.5-he0dffb1_0 ...
installing: gmp-6.1.2-h6c8ec71_1 ...
installing: graphite2-1.3.11-h16798f4_2 ...
installing: icu-58.2-h9c2bf20_1 ...
installing: jbig-2.1-hdba287a_0 ...
installing: jpeg-9b-h024ee3a_2 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: libsodium-1.0.16-h1bed415_0 ...
installing: libtool-2.4.6-h544aabb_3 ...
installing: libxcb-1.13-h1bed415_1 ...
installing: lzo-2.10-h49e0be7_2 ...
installing: mkl-2018.0.2-1 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: patchelf-0.9-hf79760b_2 ...

In [ ]:
#creating the python 3.6.2 environment for the project.
!conda create -y --name sarcopenia-ai python=3.6.2

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.4
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /usr/local/envs/sarcopenia-ai

  added / updated specs: 
    - python=3.6.2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libedit-3.1                |       heed3624_0         171 KB
    tk-8.6.10                  |       hbc83047_0         3.2 MB
    libffi-3.2.1               |    hf484d3e_1007          5

In [ ]:
# Activate newly created pthon environment
!source activate sarcopenia-ai

In [ ]:
# Install the other packages with the right version for the project
!/usr/local/envs/sarcopenia-ai/bin/pip install -e .

Obtaining file:///content/gdrive/My%20Drive/sarcopenia-ai-master
     |████████████████████████████████| 377.0 MB 24 kB/s 
     |████████████████████████████████| 312 kB 73.4 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 2.9 MB 55.4 MB/s 
     |████████████████████████████████| 3.3 MB 69.1 MB/s 
     |████████████████████████████████| 28.3 MB 1.4 MB/s 
     |████████████████████████████████| 948 kB 61.7 MB/s 
     |████████████████████████████████| 44.9 MB 68 kB/s 
     |████████████████████████████████| 35.4 MB 87 kB/s 
     |████████████████████████████████| 14.5 MB 51.3 MB/s 
     |████████████████████████████████| 25.9 MB 89.8 MB/s 
     |████████████████████████████████| 2.9 MB 54.9 MB/s 
     |████████████████████████████████| 2.2 MB 54.6 MB/s 
     |████████████████████████████████| 88 kB 9.5 MB/s 
     |████████████████████████████████| 88 kB 9.4 MB/s 
     |████████████████████████████████| 127 kB 69.1 MB/s 
     |███████

## Data pre-processing


In [ ]:
!sudo apt-get install dcm2niix

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libyaml-cpp0.5v5
Suggested packages:
  pigz
The following NEW packages will be installed:
  dcm2niix libyaml-cpp0.5v5
0 upgraded, 2 newly installed, 0 to remove and 14 not upgraded.
Need to get 295 kB of archives.
After this operation, 1,105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libyaml-cpp0.5v5 amd64 0.5.2-4ubuntu1 [150 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 dcm2niix amd64 1.0.20171215-1 [145 kB]
Fetched 295 kB in 0s (3,292 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This f

In [ ]:
#To check if the conversion tool command line is recognized
!dcm2niix

Compression will be faster with 'pigz' installed
Chris Rorden's dcm2niiX version v1.0.20171215 (OpenJPEG build) GCC7.3.0 (64-bit Linux)
usage: dcm2niix [options] <in_folder>
 Options :
  -1..-9 : gz compression level (1=fastest..9=smallest, default 6)
  -b : BIDS sidecar (y/n/o(o=only: no NIfTI), default y)
   -ba : anonymize BIDS (y/n, default y)
  -c : comment stored as NIfTI aux_file (up to 24 characters)
  -d : diffusion volumes sorted by b-value (y/n, default n)
  -f : filename (%a=antenna  (coil) number, %c=comments, %d=description, %e echo number, %f=folder name, %i ID of patient, %j seriesInstanceUID, %k studyInstanceUID, %m=manufacturer, %n=name of patient, %p=protocol, %s=series number, %t=time, %u=acquisition number, %v=vendor, %x=study ID; %z sequence name; default '%f_%p_%t_%s')
  -h : show help
  -i : ignore derived, localizer and 2D images (y/n, default n)
  -m : merge 2D slices from same series regardless of study time, echo, coil, orientation, etc. (y/n, default n)
  -

In [ ]:
#Creating the NII file from the DCM collections.
import os
#import dicom2nifti

directory = 'Anti-PD-1_Lung_55_GB_All_Data'
count = 0
for folder in os.listdir(directory):
    print("IN FOLDER: "+folder)
    if(folder=='.DS_Store'):
        continue
    for sub_folder in os.listdir(directory+"/"+folder):
        if(sub_folder=='.DS_Store'):
            continue
        #print("IN SUB-FOLDER: "+sub_folder)
        for sub_sub_folder in os.listdir(directory+"/"+folder+"/"+sub_folder):
            if(sub_sub_folder=='.DS_Store'):
                continue
            print("IN SUB-SUB-FOLDER: "+sub_sub_folder)
            command = 'dcm2niix -f "%f_%p_%t_%s" -p y -b n -m y "'+directory+"/"+folder+"/"+sub_folder+"/"+sub_sub_folder+'"'
            #print(command)
            count = count+1
            returned_value = os.system(command)  # returns the exit code in unix
            #print('returned value:', returned_value)
            print(count)
print("Count: "+str(count))

IN FOLDER: PD-1-Lung-00007
IN SUB-SUB-FOLDER: 4.000000-ST N141216 A2030-49778
1
IN SUB-SUB-FOLDER: 602.000000-CH SAG-79081
2
IN SUB-SUB-FOLDER: 2.000000-SCOUT - CAP-65180
3
IN SUB-SUB-FOLDER: 5.000000-LUNG N14 A30-15097
4
IN SUB-SUB-FOLDER: 1.000000-SCOUT - PELVIS-33616
5
IN SUB-SUB-FOLDER: 601.000000-CH COR-93198
6
IN SUB-SUB-FOLDER: 604.000000-MIP LUNG-29410
7
IN SUB-SUB-FOLDER: 3.000000-ST N141216 A0-16674
8
IN SUB-SUB-FOLDER: 606.000000-ABD COR-13415
9
IN SUB-SUB-FOLDER: 607.000000-ABD SAG-59162
10
IN SUB-SUB-FOLDER: 3.000000-CT Soft-50740
11
IN SUB-SUB-FOLDER: 4.000000-CT Lung-94863
12
IN SUB-SUB-FOLDER: 2.000000-CTAC-71884
13
IN SUB-SUB-FOLDER: 1.000000-Scout-74301
14
IN FOLDER: PD-1-Lung-00001
IN SUB-SUB-FOLDER: 1.000000-Scout-63493
15
IN SUB-SUB-FOLDER: 3.000000-CT Soft-93141
16
IN SUB-SUB-FOLDER: 2.000000-CTAC-07635
17
IN SUB-SUB-FOLDER: 4.000000-CT Lung-78716
18
IN SUB-SUB-FOLDER: 200.000000-Smart Prep Series-85726
19
IN SUB-SUB-FOLDER: 3.000000-ST N141215 A2030-37275
20
IN S

## Executing the algorithm against a folder of DCM collections
### by executing the python file mycode.py with the DCM collection folders as parameter -i 
### The provided folder should contain the (subfolders) studies, each of those contains the (sub-subfolders) series. A serie is a collection (folder) containing one or more DCM files.

In [ ]:
# This cell is an illustration on how to execute the pthon code.
# You might need to copy the following line and adjust it as it fit to your specific dataset location.

In [ ]:
!/usr/local/envs/sarcopenia-ai/bin/pip install pandas

     |████████████████████████████████| 9.5 MB 13.5 MB/s 
     |████████████████████████████████| 509 kB 48.4 MB/s 


In [ ]:
!/usr/local/envs/sarcopenia-ai/bin/python mycode.py -i github_dataset

/usr/local/envs/sarcopenia-ai/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/envs/sarcopenia-ai/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/envs/sarcopenia-ai/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/envs/sarcopenia-ai/lib/python3.6/site-packages/ten

In [ ]:
!/usr/local/envs/sarcopenia-ai/bin/python mycode_v3.py -i Anti-PD-1_Lung_55_GB_All_Data
#TCGA-LUAD
#test_data

/usr/local/envs/sarcopenia-ai/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/envs/sarcopenia-ai/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/envs/sarcopenia-ai/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/envs/sarcopenia-ai/lib/python3.6/site-packages/ten

In [ ]:
!/usr/local/envs/sarcopenia-ai/bin/python sarcopenia_ai/apps/slice_detection/predict.py --cuda_device 0 --image volume.nii.gz

Traceback (most recent call last):
  File "sarcopenia_ai/apps/slice_detection/predict.py", line 1, in <module>
  File "/usr/local/envs/sarcopenia-ai/lib/python3.6/site-packages/SimpleITK/__init__.py", line 19, in <module>
    from SimpleITK.extra import *
  File "/usr/local/envs/sarcopenia-ai/lib/python3.6/site-packages/SimpleITK/extra.py", line 118, in <module>
    import numpy
  File "/usr/local/envs/sarcopenia-ai/lib/python3.6/site-packages/numpy/__init__.py", line 223, in <module>
    from .testing import Tester
  File "/usr/local/envs/sarcopenia-ai/lib/python3.6/site-packages/numpy/testing/__init__.py", line 8, in <module>
    from unittest import TestCase
  File "/usr/local/envs/sarcopenia-ai/lib/python3.6/unittest/__init__.py", line 59, in <module>
    from .case import (TestCase, FunctionTestCase, SkipTest, skip, skipIf,
  File "/usr/local/envs/sarcopenia-ai/lib/python3.6/unittest/case.py", line 6, in <module>
    import logging
  File "/usr/local/envs/sarcopenia-ai/lib/python3